# Indexing and Selection

Now that we are familiar with pandas' data structures, we can turn our attention to some of the intermediate features of data frames, which include:
    
- Intelligent label-based **slicing, fancy indexing, and subsetting** of large data sets
- **Hierarchical labeling** of axes
- **Sorting and ranking** of data in DataFrames
- Easy handling of **missing data**
- Data **summarization** tools

In this section, we will manipulate data collected from ocean-going vessels on the eastern seaboard. Vessel operations are monitored using the **Automatic Identification System (AIS)**, a safety at sea navigation technology which vessels are required to maintain and that uses transponders to transmit very high frequency (VHF) radio signals containing static information including ship name, call sign, and country of origin, as well as dynamic information unique to a particular voyage such as vessel location, heading, and speed. 

![AIS](images/ais.gif)

The International Maritime Organization’s (IMO) International Convention for the Safety of Life at Sea requires functioning AIS capabilities on all vessels 300 gross tons or greater and the US Coast Guard requires AIS on nearly all vessels sailing in U.S. waters. The Coast Guard has established a national network of AIS receivers that provides coverage of nearly all U.S. waters. **AIS signals** are transmitted several times each minute and the network is capable of handling thousands of reports per minute and updates as often as every two seconds. Therefore, a typical voyage in our study might include the transmission of hundreds or thousands of AIS encoded signals. This provides a rich source of spatial data that includes both **spatial and temporal information**.

For our purposes, we will use **summarized data** that describes the transit of a given vessel through a particular administrative area. The data includes the start and end time of the transit segment, as well as information about the speed of the vessel, how far it travelled, etc.

In [8]:
import pandas as pd
import numpy as np

vessels = pd.read_csv('../data/AIS/vessel_information.csv', index_col=0)

In [9]:
vessels.shape

(10771, 10)

## Indexing and Selection

Indexing works analogously to indexing in NumPy arrays, except we can use the labels in the `Index` object to extract values in addition to arrays of integers.

In [12]:
vessels.columns

Index(['num_names', 'names', 'sov', 'flag', 'flag_type', 'num_loas', 'loa',
       'max_loa', 'num_types', 'type'],
      dtype='object')

In [13]:
# Sample Series object
flag = vessels.flag
flag

mmsi
1                                               Unknown
9                                               Unknown
21                                              Unknown
74                                              Unknown
103                                             Unknown
310                                            Bermuda 
3011                                          Anguilla 
4731                               Yemen (Republic of) 
15151                                           Unknown
46809                             Syrian Arab Republic 
80404                                           Unknown
82003                                           Unknown
298716                                          Unknown
366235                        United States of America 
439541                                          Unknown
453556                                          Unknown
505843                                          Unknown
527918                                     

In [14]:
# Numpy-style indexing
flag[:10]

mmsi
1                      Unknown
9                      Unknown
21                     Unknown
74                     Unknown
103                    Unknown
310                   Bermuda 
3011                 Anguilla 
4731      Yemen (Republic of) 
15151                  Unknown
46809    Syrian Arab Republic 
Name: flag, dtype: object

In [15]:
# Indexing by label
flag[[298716,725011300]]

mmsi
298716       Unknown
725011300     Chile 
Name: flag, dtype: object

In a `DataFrame` we can slice along either or both axes:

In [16]:
vessels[['num_names','num_types']].head()

,num_names,num_types
mmsi,,
1,8,4
9,3,2
21,1,1
74,2,1
103,3,2


In [18]:
vessels[vessels.max_loa>700]

,num_names,names,sov,flag,flag_type,num_loas,loa,max_loa,num_types,type
mmsi,,,,,,,,,,
211311970,2,Antwerpen Express/Dallas Express,N,Germany (Federal Republic of),Foreign,3,294.0/512.0/806.0,806.0,1,Cargo
218292000,1,Seapike,N,Germany (Federal Republic of),Foreign,3,199.0/200.0/711.0,711.0,1,Tanker
235760000,1,Methane Princess,N,United Kingdom of Great Britain and Northern I...,Foreign,2,277.0/789.0,789.0,1,Tanker
354092000,1,Msc Laura,N,Panama (Republic of),Foreign,3,300.0/301.0/812.0,812.0,1,Cargo
538090124,1,Pacific Destiny,N,Marshall Islands (Republic of the),Foreign,2,196.0/708.0,708.0,1,Cargo


The indexing field `loc` allows us to select subsets of rows and columns in an intuitive way:

In [19]:
vessels.loc[720768000, ['names','flag', 'type']]

names             Rusty Pelican
flag     Bolivia (Republic of) 
type                     Towing
Name: 720768000, dtype: object

In [20]:
vessels.loc[:4731, 'names']

mmsi
1       Bil Holman Dredge/Dredge Capt Frank/Emo/Offsho...
9                              000000009/Raven/Shearwater
21                                          Us Gov Vessel
74                                      Mcfaul/Sarah Bell
103              Ron G/Us Navy Warship 103/Us Warship 103
310                                              Arabella
3011                                           Charleston
4731                                            000004731
Name: names, dtype: object

Slicing also works with string variables, since an index has an intrinsic order, regardless of label:

In [22]:
vessels.columns

Index(['num_names', 'names', 'sov', 'flag', 'flag_type', 'num_loas', 'loa',
       'max_loa', 'num_types', 'type'],
      dtype='object')

In [21]:
vessels.loc[:310, 'flag':'loa']

,flag,flag_type,num_loas,loa
mmsi,,,,
1,Unknown,Unknown,7,42.0/48.0/57.0/90.0/138.0/154.0/156.0
9,Unknown,Unknown,2,50.0/62.0
21,Unknown,Unknown,1,208.0
74,Unknown,Unknown,1,155.0
103,Unknown,Unknown,2,26.0/155.0
310,Bermuda,Foreign,1,47.0


In addition to using `loc` to select rows and columns by **label**, pandas also allows indexing by **position** using the `iloc` attribute.

So, we can query rows and columns by absolute position, rather than by name:

In [23]:
vessels.iloc[:5, 5:8]

,num_loas,loa,max_loa
mmsi,,,
1,7,42.0/48.0/57.0/90.0/138.0/154.0/156.0,156.0
9,2,50.0/62.0,62.0
21,1,208.0,208.0
74,1,155.0,155.0
103,2,26.0/155.0,155.0


### Exercise

You can use the `isin` method query a DataFrame based upon a list of values as follows: 

    data['color'].isin(['red', 'blue'])

Use `isin` to find all ships that registered in either Denmark or Japan. How many records contain these values?

In [30]:
vessels['flag'] = vessels.flag.str.strip()

In [39]:
vessels.flag.isin(['Denmark', 'Japan'])

mmsi
1            False
9            False
21           False
74           False
103          False
310          False
3011         False
4731         False
15151        False
46809        False
80404        False
82003        False
298716       False
366235       False
439541       False
453556       False
505843       False
527918       False
565026       False
572329       False
587370       False
641114       False
642262       False
693559       False
883085       False
1193046      False
1193946      False
1233916      False
1239468      False
3041300      False
             ...  
720728000    False
720754000    False
720768000    False
725004700    False
725005560    False
725005570    False
725011300    False
725018300    False
725019006    False
725021000    False
725022000    False
730000161    False
730010001    False
730026000    False
730031000    False
735057548    False
735059037    False
760101000    False
770576100    False
812719000    False
857632392    False
8669468

In [ ]:
vessels.

## Indexing with `where`

Pandas `DataFrame` objects also posess a `where` index for indexing that returns the values that satisfy the condition, but retain the index of the original `DataFrame`, so that the shape does not change. This is important when **alignment** is required for operations between `DataFrame`s.

In [40]:
np.random.seed(42)
normal_vals = pd.DataFrame({'x{}'.format(i):np.random.randn(100) for i in range(5)})

normal_vals.head()

,x0,x1,x2,x3,x4
0,0.496714,-1.415371,0.357787,-0.828995,-1.594428
1,-0.138264,-0.420645,0.560785,-0.560181,-0.599375
2,0.647689,-0.342715,1.083051,0.747294,0.005244
3,1.523030,-0.802277,1.053802,0.610370,0.046981
4,-0.234153,-0.161286,-1.377669,-0.020902,-0.450065


In [41]:
normal_vals.where(normal_vals>0).head()

,x0,x1,x2,x3,x4
0,0.496714,NaN,0.357787,NaN,NaN
1,NaN,NaN,0.560785,NaN,NaN
2,0.647689,NaN,1.083051,0.747294,0.005244
3,1.523030,NaN,1.053802,0.610370,0.046981
4,NaN,NaN,NaN,NaN,NaN


`where` includes an optional `other` argument that accepts a scalar or tabular values (or a callable) to replace values in the `DataFrame` that do not satisfy the condition.

For example, we can use this to return the absolute values of `X`:

In [42]:
normal_vals.where(normal_vals>0, other=-normal_vals).head()

,x0,x1,x2,x3,x4
0,0.496714,1.415371,0.357787,0.828995,1.594428
1,0.138264,0.420645,0.560785,0.560181,0.599375
2,0.647689,0.342715,1.083051,0.747294,0.005244
3,1.523030,0.802277,1.053802,0.610370,0.046981
4,0.234153,0.161286,1.377669,0.020902,0.450065


Similarly, a callable can be used when we need to modify the replaced value:

In [43]:
normal_vals.where(normal_vals>0, other=lambda y: -y*100).head()

,x0,x1,x2,x3,x4
0,0.496714,141.537074,0.357787,82.899501,159.442766
1,13.826430,42.064532,0.560785,56.018104,59.937502
2,0.647689,34.271452,1.083051,0.747294,0.005244
3,1.523030,80.227727,1.053802,0.610370,0.046981
4,23.415337,16.128571,137.766937,2.090159,45.006547


Conversely, `mask` is the inverse boolean of `where`:

In [44]:
normal_vals.mask(normal_vals>0).head()

,x0,x1,x2,x3,x4
0,NaN,-1.415371,NaN,-0.828995,-1.594428
1,-0.138264,-0.420645,NaN,-0.560181,-0.599375
2,NaN,-0.342715,NaN,NaN,NaN
3,NaN,-0.802277,NaN,NaN,NaN
4,-0.234153,-0.161286,-1.377669,-0.020902,-0.450065


## Selection with `query`

At times, selection using indexing can be verbose because it requires repeated use of the `DataFrame` namespace.

In [45]:
normal_vals[(normal_vals.x1 > normal_vals.x0) & (normal_vals.x3 > normal_vals.x2)].head()

,x0,x1,x2,x3,x4
4,-0.234153,-0.161286,-1.377669,-0.020902,-0.450065
5,-0.234137,0.404051,-0.937825,0.117327,0.622850
6,1.579213,1.886186,0.515035,1.277665,-1.067620
8,-0.469474,0.257550,0.515048,0.547097,0.120296
13,-1.913280,2.463242,0.651391,0.813510,1.277677


For a more concise (and readable) syntax, we can use the new `query` method to perform selection on a `DataFrame`. Instead of having to type the fully-specified column, we can simply pass a string that describes what to select. The query above is then simply:

In [46]:
normal_vals.query('(x1 > x0) & (x3 > x2)').head()

,x0,x1,x2,x3,x4
4,-0.234153,-0.161286,-1.377669,-0.020902,-0.450065
5,-0.234137,0.404051,-0.937825,0.117327,0.622850
6,1.579213,1.886186,0.515035,1.277665,-1.067620
8,-0.469474,0.257550,0.515048,0.547097,0.120296
13,-1.913280,2.463242,0.651391,0.813510,1.277677


The `DataFrame.index` and `DataFrame.columns` are placed in the query namespace by default. If you want to refer to a variable in the current namespace, you can prefix the variable with `@`:

In [47]:
min_loa = 700

In [48]:
vessels.query('max_loa > @min_loa')

,num_names,names,sov,flag,flag_type,num_loas,loa,max_loa,num_types,type
mmsi,,,,,,,,,,
211311970,2,Antwerpen Express/Dallas Express,N,Germany (Federal Republic of),Foreign,3,294.0/512.0/806.0,806.0,1,Cargo
218292000,1,Seapike,N,Germany (Federal Republic of),Foreign,3,199.0/200.0/711.0,711.0,1,Tanker
235760000,1,Methane Princess,N,United Kingdom of Great Britain and Northern I...,Foreign,2,277.0/789.0,789.0,1,Tanker
354092000,1,Msc Laura,N,Panama (Republic of),Foreign,3,300.0/301.0/812.0,812.0,1,Cargo
538090124,1,Pacific Destiny,N,Marshall Islands (Republic of the),Foreign,2,196.0/708.0,708.0,1,Cargo
